In [3]:
#!pip install category_encoders

     |████████████████████████████████| 80 kB 3.2 MB/s eta 0:00:011


In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from lightgbm import LGBMClassifier

import category_encoders as ce


In [5]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
sample = pd.read_csv('submit_sample.csv', header=None)

In [6]:
train.info(), train.shape, test.shape, sample.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27100 entries, 0 to 27099
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        27100 non-null  int64 
 1   job        27100 non-null  object
 2   marital    27100 non-null  object
 3   education  27100 non-null  object
 4   default    27100 non-null  object
 5   balance    27100 non-null  int64 
 6   housing    27100 non-null  object
 7   loan       27100 non-null  object
 8   contact    27100 non-null  object
 9   day        27100 non-null  int64 
 10  month      27100 non-null  object
 11  duration   27100 non-null  int64 
 12  campaign   27100 non-null  int64 
 13  pdays      27100 non-null  int64 
 14  previous   27100 non-null  int64 
 15  poutcome   27100 non-null  object
 16  y          27100 non-null  int64 
dtypes: int64(8), object(9)
memory usage: 3.7+ MB


(None, (27100, 17), (18050, 16), (18050, 2))

In [7]:
test['y'] = -999

In [8]:
all_df = pd.concat([train, test], axis=0)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45150 entries, 0 to 18049
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45150 non-null  int64 
 1   job        45150 non-null  object
 2   marital    45150 non-null  object
 3   education  45150 non-null  object
 4   default    45150 non-null  object
 5   balance    45150 non-null  int64 
 6   housing    45150 non-null  object
 7   loan       45150 non-null  object
 8   contact    45150 non-null  object
 9   day        45150 non-null  int64 
 10  month      45150 non-null  object
 11  duration   45150 non-null  int64 
 12  campaign   45150 non-null  int64 
 13  pdays      45150 non-null  int64 
 14  previous   45150 non-null  int64 
 15  poutcome   45150 non-null  object
 16  y          45150 non-null  int64 
dtypes: int64(8), object(9)
memory usage: 6.2+ MB


In [9]:
all_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,31,services,married,secondary,no,12294,yes,no,cellular,21,nov,101,3,498,0,other,0
1,29,entrepreneur,single,tertiary,no,43027,no,no,cellular,22,aug,158,2,702,0,unknown,1
2,35,management,married,tertiary,no,12252,yes,no,cellular,11,nov,351,1,826,0,failure,0
3,31,technician,married,secondary,no,99121,yes,yes,unknown,16,may,658,2,120,0,failure,0
4,48,unemployed,married,primary,no,42005,yes,no,telephone,3,apr,177,1,273,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,self-employed,married,tertiary,no,98357,yes,no,cellular,6,jul,101,2,417,0,failure,-999
18046,34,blue-collar,married,secondary,no,29621,yes,no,cellular,12,may,345,1,815,0,unknown,-999
18047,34,admin.,single,secondary,no,94260,yes,no,unknown,16,may,121,2,370,0,unknown,-999


In [10]:
all_df.isnull().any()

age          False
job          False
marital      False
education    False
default      False
balance      False
housing      False
loan         False
contact      False
day          False
month        False
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
y            False
dtype: bool

In [11]:
# One Hot Encoding
ce_ohe = ce.OneHotEncoder(handle_unknown='impute')
ce_ohe.fit(all_df.drop('y', axis=1))


OneHotEncoder(cols=['job', 'marital', 'education', 'default', 'housing', 'loan',
                    'contact', 'month', 'poutcome'],
              handle_unknown='impute')

In [12]:
X = ce_ohe.transform(all_df[all_df['y'] != -999].drop('y', axis=1))
y = all_df[all_df['y'] != -999]['y']

In [13]:
X.shape, y.shape

((27100, 50), (27100,))

In [22]:
parameters = {
    'max_depth': list(range(3,15, 1)),
    'random_state': [0],
}

In [23]:
## lightgbm gridsearch cross validation

lgb = LGBMClassifier()

gcv = GridSearchCV(lgb, parameters, cv=5, scoring='roc_auc', n_jobs=-1, return_train_score=True)
gcv.fit(X, y)

GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                         'random_state': [0]},
             return_train_score=True, scoring='roc_auc')

In [24]:
gcv.best_params_

{'max_depth': 5, 'random_state': 0}

In [38]:
importance = pd.DataFrame(gcv.best_estimator_.feature_importances_, index=X.columns)
importance

,0
age,220
job_1,12
job_2,18
job_3,12
job_4,32
job_5,5
job_6,12
job_7,18
job_8,17
job_9,2


In [40]:
import seaborn as sns
sns.barplot(data=importance, )

ValueError: Could not interpret input '0'

In [ ]:
feature_importances = pd.DataFrame()
feature_importances['feature'] = features
feature_importances['importance'] = abs(corrs)
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

plt.figure(figsize=(16, 8))
sns.barplot(data=feature_importances.head(10), x='importance', y='feature')

In [20]:
## roc_auc
y_pred = gcv.predict(X)

roc_auc_score(y, y_pred)

## No.1. score : 0.6329422213571606

0.6299245986558396

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y, y_pred)
auc = metrics.auc(fpr, tpr)
print(auc)

In [ ]:
testX = ce_ohe.transform(all_df[all_df['y'] == -999].drop('y', axis=1))
testX

In [ ]:
pred = gcv.predict_proba(testX)
pred = pred[:,1]
pred

In [ ]:
sample[1] = pred
sample.to_csv('submit_20200809_02.csv', index=None, header=None)